In [1]:
import requests
import pandas as pd

## 1. Use the API to get all available data for the _GDP per capita, PPP (constant 2017 international $)_ indicator. 
Hint: this indicator has code "NY.GDP.PCAP.PP.KD". Adjust the query parameters so that you can retrieve all available rows. Convert the results to a DataFrame.

In [ ]:
endpoint = 'https://api.worldbank.org/V2/country/all/indicator/NY.GDP.PCAP.PP.KD'

In [ ]:
params = {
    'per_page': 20000,
    'format': 'json'
}

In [ ]:
response = requests.get(endpoint, params = params)

In [ ]:
response

In [ ]:
response.json()

In [ ]:
res = response.json()[1]

In [ ]:
res

In [ ]:
gdp_df = pd.json_normalize(res)

In [ ]:
gdp_df.head()

## 2. Now, use the API to get all available data for _Life expectancy at birth, total (years)_. 
This indicator has code "SP.DYN.LE00.IN". Again, convert the results to a DataFrame.

In [ ]:
endpoint2 = 'https://api.worldbank.org/V2/country/all/indicator/SP.DYN.LE00.IN'

In [ ]:
response2 = requests.get(endpoint2, params)

In [ ]:
response2

In [ ]:
response2.json()

In [ ]:
res2 = response2.json()[1]

In [ ]:
life_expectancy_df = pd.json_normalize(res2)

In [ ]:
life_expectancy_df.head()

## 3. Merge the two results DataFrames together. You may want to rename or drop columns prior to merging.

In [ ]:
gdp_df = gdp_df.rename(columns = {'value': 'gdp_per_capita'})[['country.value', 'date', 'gdp_per_capita']]

In [ ]:
gdp_df.head()

In [ ]:
life_expectancy_df = life_expectancy_df.rename(columns = {'value': 'life_expectancy'})[['country.value', 'date', 'life_expectancy']]

In [ ]:
life_expectancy_df.head()

In [ ]:
indicators_df = pd.merge(gdp_df, life_expectancy_df, on = ['country.value', 'date'], how = 'outer').rename(columns = {'date': 'year', 'country.value': 'country'})

In [ ]:
indicators_df.head()

## 4. You can also get more information about the available countries (region, capital city, income level classification, etc.) by using the [Country API](https://datahelpdesk.worldbank.org/knowledgebase/articles/898590-country-api-queries). 
Use this API to pull in all available data. Merge this with your other datasets. Use this to now remove the rows that correspond to regions and not countries.

In [ ]:
endpoint4 = 'https://api.worldbank.org/v2/country'

In [ ]:
response4 = requests.get(endpoint4, params = params)

In [ ]:
response4

In [ ]:
res4 = response4.json()[1]

In [ ]:
countries_df = pd.json_normalize(res4)[['name', 'region.value']].rename(columns = {'name': 'country', 'region.value': 'region'})

In [ ]:
countries_df.head()

In [ ]:
indicators_df = pd.merge(indicators_df, countries_df, on = 'country', how = 'left')

In [ ]:
country_indicators = indicators_df.loc[indicators_df['region'] != 'Aggregates']

In [ ]:
country_indicators.head()

**Bonus Questions:** After doing a basic request to get all records, you can attempt these exercises to get additional practice using query parameters.

## 1. Adjust your request so that it returns data just for the United States.

2. Adjust your request so that it returns data just for the United States for the year 2021.

3. Adjust your request so that it returns data just for the United States for the years 2000 through 2021.

4. Adjust your request so that it returns data for the United States and Canada for the years 2000 through 2021.

In [ ]:
def un_data(country, indicator, date):
    endpoint = 'https://api.worldbank.org/V2/country/' + country + '/indicator/' + indicator + '?date=' + date
    response = requests.get(endpoint, params = {'per_page': 20000, 'format': 'json'})
    response_df = pd.json_normalize(response.json()[1])

    return response_df
    

In [ ]:
un_data('usa', 'SP.DYN.LE00.IN', '2021')

In [ ]:
un_data('usa;can', 'SP.DYN.LE00.IN', '2000:2021')

## 5. If you haven't already done so and you would like to get some additional practice using loops, use the page parameter in order to pull all records. 
Do not change the value of the per_page parameter. You will likely need to utilize a loop of some kind in order to pull all records.

In [3]:
## adjust these three variables to change the data retrieved
endpoint = 'https://api.worldbank.org/V2/'
country = 'all'
indicator = 'NY.GDP.PCAP.PP.KD'

## don't touch these
URL = endpoint + 'country/' + country + '/indicator/' + indicator
pages = requests.get(URL, params = {'format': 'json'}).json()[0]['pages']


for page in range(1, pages+1):
    response = requests.get(URL, params = {'format': 'json', 'page': page}).json()[1]
    loop_df = pd.json_normalize(response)

    if page == 1:
        df = loop_df
    else:
        df = pd.concat([df, loop_df])

    print(page)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


C:\Users\5pira\AppData\Local\Temp\ipykernel_26064\692669998.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, loop_df])


46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
30

In [5]:
df.reset_index(drop = True)

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,3968.963751,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
1,AFE,2023,3948.142721,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
2,AFE,2022,3974.244214,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
3,AFE,2021,3933.580905,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
4,AFE,2020,3861.068816,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZH,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...,...
17285,ZWE,1964,NaN,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZW,Zimbabwe
17286,ZWE,1963,NaN,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZW,Zimbabwe
17287,ZWE,1962,NaN,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZW,Zimbabwe
17288,ZWE,1961,NaN,,,0,NY.GDP.PCAP.PP.KD,"GDP per capita, PPP (constant 2021 internation...",ZW,Zimbabwe


## 6. The endpoint "http://api.worldbank.org/v2/indicator" will return information about all available indicators. 
See [Indicator API Queries](https://datahelpdesk.worldbank.org/knowledgebase/articles/898599-indicator-api-queries). Use this endpoint to find the code for the "Public Expenditure on Education (% GDP)" indicator.

In [ ]:
indicators_list = requests.get('http://api.worldbank.org/v2/indicator', params = {'per_page': 30000, 'format': 'json'}).json()[1]

In [ ]:
indicators_df = pd.json_normalize(indicators_list)

In [ ]:
indicators_df.loc[indicators_df['name'].str.lower().str.contains('public expenditure on education'), 'id'].values[0]